# eSure Data Science Brief
_______

### Executive summary
Given a home insurance dataset covering 2007 - 2012, this notebook details an exploration of the dataset and development of a model that predicts future lapsed cases.

_______

### 0. Import modules

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import logging

In [3]:
#_base_format = "[%(levelname)s] | %(asctime)s | %(name)s | %(message)s"
_base_format = ""
logging.basicConfig(format = _base_format,
filename = 'output.log', encoding = 'utf-8', level = logging.DEBUG)

# 1. Data exploration

In [4]:
data_path = "../data/raw/home_insurance.csv"
df = pd.read_csv(data_path, header=0)

In [5]:
df.head()

,QUOTE_DATE,COVER_START,CLAIM3YEARS,P1_EMP_STATUS,P1_PT_EMP_STATUS,BUS_USE,CLERICAL,AD_BUILDINGS,RISK_RATED_AREA_B,SUM_INSURED_BUILDINGS,...,HP3_ADDON_PRE_REN,HP3_ADDON_POST_REN,MTA_FLAG,MTA_FAP,MTA_APRP,MTA_DATE,LAST_ANN_PREM_GROSS,POL_STATUS,i,Police
0,11/22/2007,22/11/2007,N,R,NaN,N,NaN,Y,19.0,1000000.0,...,N,N,N,NaN,NaN,NaN,274.81,Lapsed,1,P000001
1,11/22/2007,01/01/2008,N,E,NaN,Y,N,Y,25.0,1000000.0,...,N,N,Y,308.83,-9.27,NaN,308.83,Live,2,P000002
2,11/23/2007,23/11/2007,N,E,NaN,N,NaN,N,NaN,0.0,...,N,N,Y,52.65,52.65,03/11/2010,52.65,Live,3,P000003
3,11/23/2007,12/12/2007,N,R,NaN,N,NaN,N,NaN,0.0,...,N,N,N,NaN,NaN,NaN,54.23,Live,4,P000004
4,11/22/2007,15/12/2007,N,R,NaN,N,NaN,Y,5.0,1000000.0,...,N,N,N,NaN,NaN,NaN,244.58,Live,5,P000005


In [6]:
def log_df(df: "pd.DataFrame") -> None:
    """
    """
    logging.info("")
    logging.info("Pre-processing -> Understanding the data...")
    logging.info(f"Length of dataframe: {len(df)}")
    logging.info("| "+"COLUMN".rjust(23)+"|    DTYPE".rjust(11)+
    " | % NULL".rjust(8)+
    " | UNIQUE VALUES LENGTH | ".rjust(10)+
    "SAMPLE (display all unique values if len < 10)".rjust(20))
    for i in df.columns:
        num_nulls = round(df[i].isnull().mean()*100,1)
        if len(df[i].unique()) < 10:
            logging.info(f"| {i:>23} | {str(df.dtypes[i]):>8} | {num_nulls:>6} | {len(df[i].unique()):>20} | {df[i].unique()}")
        else:
            logging.info(f"| {i:>23} | {str(df.dtypes[i]):>8} | {num_nulls:>6} | {len(df[i].unique()):>20} | {df[i][df[i].notna()].sample(2).values}")
    logging.info("")
    return None

In [7]:
log_df(df)

In [8]:
df[df["PAYMENT_FREQUENCY"].notna()].PAYMENT_METHOD.value_counts()

NonDD       38593
PureDD      38552
DD-Other     3409
Name: PAYMENT_METHOD, dtype: int64

Analysing the above `output.log` gives an insight into how to process the data. In the appendix section, the variable names are included with their descriptions. 

Some columns can be dropped in this first batch of pre-processing, summarised with the reason below:

| *Column*         | *Reason to be dropped* |
| -: | -: |
| i           |    All values are unique    |
| Police      |   All values are unique     |
| CAMPAIGN_DESC      |   All values are null     |
| PAYMENT_FREQUENCY      |   Either NaN (68%) or 1.  |




In [9]:
df.drop(["i", "Police", "CAMPAIGN_DESC", "PAYMENT_FREQUENCY"],
axis = 1, inplace = True)
df.head()

,QUOTE_DATE,COVER_START,CLAIM3YEARS,P1_EMP_STATUS,P1_PT_EMP_STATUS,BUS_USE,CLERICAL,AD_BUILDINGS,RISK_RATED_AREA_B,SUM_INSURED_BUILDINGS,...,HP2_ADDON_PRE_REN,HP2_ADDON_POST_REN,HP3_ADDON_PRE_REN,HP3_ADDON_POST_REN,MTA_FLAG,MTA_FAP,MTA_APRP,MTA_DATE,LAST_ANN_PREM_GROSS,POL_STATUS
0,11/22/2007,22/11/2007,N,R,NaN,N,NaN,Y,19.0,1000000.0,...,N,N,N,N,N,NaN,NaN,NaN,274.81,Lapsed
1,11/22/2007,01/01/2008,N,E,NaN,Y,N,Y,25.0,1000000.0,...,N,N,N,N,Y,308.83,-9.27,NaN,308.83,Live
2,11/23/2007,23/11/2007,N,E,NaN,N,NaN,N,NaN,0.0,...,N,N,N,N,Y,52.65,52.65,03/11/2010,52.65,Live
3,11/23/2007,12/12/2007,N,R,NaN,N,NaN,N,NaN,0.0,...,N,N,N,N,N,NaN,NaN,NaN,54.23,Live
4,11/22/2007,15/12/2007,N,R,NaN,N,NaN,Y,5.0,1000000.0,...,N,N,N,N,N,NaN,NaN,NaN,244.58,Live


## 1.1 Missing data - Identifying and handling

Lots of the features in `output.log` list a null percentage of 26.2% which warrants further investigation. 

In [10]:
logging.info("Pre-processing -> Removing POL_STATUS = null")

mask = df["POL_STATUS"].notnull()

log_df(df[mask])


As shown in `output.log`, removing the rows where `POL_STATUS = null` removed 

In [17]:
df.drop(df[~mask].index, axis = 0, inplace = True) # drop null values from mask


In [18]:
df.head()

,QUOTE_DATE,COVER_START,CLAIM3YEARS,P1_EMP_STATUS,P1_PT_EMP_STATUS,BUS_USE,CLERICAL,AD_BUILDINGS,RISK_RATED_AREA_B,SUM_INSURED_BUILDINGS,...,HP2_ADDON_PRE_REN,HP2_ADDON_POST_REN,HP3_ADDON_PRE_REN,HP3_ADDON_POST_REN,MTA_FLAG,MTA_FAP,MTA_APRP,MTA_DATE,LAST_ANN_PREM_GROSS,POL_STATUS
0,11/22/2007,22/11/2007,N,R,NaN,N,NaN,Y,19.0,1000000.0,...,N,N,N,N,N,NaN,NaN,NaN,274.81,Lapsed
1,11/22/2007,01/01/2008,N,E,NaN,Y,N,Y,25.0,1000000.0,...,N,N,N,N,Y,308.83,-9.27,NaN,308.83,Live
2,11/23/2007,23/11/2007,N,E,NaN,N,NaN,N,NaN,0.0,...,N,N,N,N,Y,52.65,52.65,03/11/2010,52.65,Live
3,11/23/2007,12/12/2007,N,R,NaN,N,NaN,N,NaN,0.0,...,N,N,N,N,N,NaN,NaN,NaN,54.23,Live
4,11/22/2007,15/12/2007,N,R,NaN,N,NaN,Y,5.0,1000000.0,...,N,N,N,N,N,NaN,NaN,NaN,244.58,Live


In [ ]:
df.drop(["CLERICAL", "P1_PT_EMP_STATUS", ],
axis = 1, inplace = True)
df.head()

Summary of additional data dropped. 

| *Row*         | *Column*         | *Reason to be dropped* |
| -: | -: | -: |
| POL_STATUS = null |   ALL   |       |
| ALL |   CLERICAL   |   ~98% null values    |
| ALL |   P1_PT_EMP_STATUS   |   ~98% null values    |


XGBoost only allows `int`, `float`, or `boolean` data types.

# 2. Data Modelling

# 3. Conclusions

### Further work
Clarification on data to be used. Specifically those of an ethical concern, can we use P1_Sex/Age/marital status for predictive analytics?

# Appendices

#### Variable Description
Below taken from https://www.kaggle.com/datasets/ycanario/home-insurance
 - QUOTE_DATE: Day where the quotation was made
 - COVER_START: Beginning of the cover payment
 - CLAIM3YEARS: 3 last years loss
 - P1EMPSTATUS: Client's professional status
 - P1PTEMP_STATUS: Client's part-time professional status
 - BUS_USE: Commercial use indicator
 - CLERICAL: Administration office usage indicator
 - AD_BUILDINGS: Building coverage - Self damage
 - RISKRATEDAREA_B: Geographical Classification of Risk - Building
 - SUMINSUREDBUILDINGS: Assured Sum - Building
 - NCDGRANTEDYEARS_B: Bonus Malus - Building
 - AD_CONTENTS: Coverage of personal items - Self Damage
 - RISKRATEDAREA_C: Geographical Classification of Risk - Personal Objects
 - SUMINSUREDCONTENTS: Assured Sum - Personal Items
 - NCDGRANTEDYEARS_C: Malus Bonus - Personal Items
 - CONTENTS_COVER: Coverage - Personal Objects indicator
 - BUILDINGS_COVER: Cover - Building indicator
 - SPECSUMINSURED: Assured Sum - Valuable Personal Property
 - SPECITEMPREM: Premium - Personal valuable items
 - UNSPECHRPPREM: Unknown
 - P1_DOB: Date of birth of the client
 - P1MARSTATUS: Marital status of the client
 - P1POLICYREFUSED: Police Emission Denial Indicator
 - P1_SEX: customer sex
 - APPR_ALARM: Appropriate alarm
 - APPR_LOCKS: Appropriate lock
 - BEDROOMS: Number of bedrooms
 - ROOF_CONSTRUCTION: Code of the type of construction of the roof
 - WALL_CONSTRUCTION: Code of the type of wall construction
 - FLOODING: House susceptible to floods
 - LISTED: National Heritage Building
 - MAXDAYSUNOCC: Number of days unoccupied
 - NEIGH_WATCH: Vigils of proximity present
 - OCC_STATUS: Occupancy status
 - OWNERSHIP_TYPE: Type of membership
 - PAYING_GUESTS: Presence of paying guests
 - PROP_TYPE: Type of property
 - SAFE_INSTALLED: Safe installs
 - SECDISCREQ: Reduction of premium for security
 - SUBSIDENCE: Subsidence indicator (relative downwards motion of the surface )
 - YEARBUILT: Year of construction
 - CAMPAIGN_DESC: Description of the marketing campaign
 - PAYMENT_METHOD: Method of payment
 - PAYMENT_FREQUENCY: Frequency of payment
 - LEGALADDONPRE_REN: Option "Legal Fees" included before 1st renewal
 - LEGALADDONPOST_REN: Option "Legal Fees" included after 1st renewal
 - HOMEEMADDONPREREN: "Emergencies" option included before 1st renewal
 - HOMEEMADDONPOSTREN: Option "Emergencies" included after 1st renewal
 - GARDENADDONPRE_REN: Option "Gardens" included before 1st renewal
 - GARDENADDONPOST_REN: Option "Gardens" included after 1st renewal
 - KEYCAREADDONPRE_REN: Option "Replacement of keys" included before 1st renewal
 - KEYCAREADDONPOST_REN: Option "Replacement of keys" included after 1st renewal
 - HP1ADDONPRE_REN: Option "HP1" included before 1st renewal
 - HP1ADDONPOST_REN: Option "HP1" included after 1st renewal
 - HP2ADDONPRE_REN: Option "HP2" included before 1st renewal
 - HP2ADDONPOST_REN: Option "HP2" included afterrenewal
 - HP3ADDONPRE_REN: Option "HP3" included before 1st renewal
 - HP3ADDONPOST_REN: Option "HP3" included after renewal
 - MTA_FLAG: Mid-Term Adjustment indicator
 - MTA_FAP: Bonus up to date of Adjustment
 - MTA_APRP: Adjustment of the premium for Mid-Term Adjustmen
 - MTA_DATE: Date of Mid-Term Adjustment
 - LASTANNPREM_GROSS: Premium - Total for the previous year
 - POL_STATUS: Police status
 - Police: Police number